#Run all

In [11]:
#@title installation
! pip install --q PyICU

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [12]:
#@title File upload
import os

titan_path = "mercuryData_ไททัน.json"
boss_path = "mercuryData_บอสตาล.json"
yok_path = "mercuryData_น้องหยก.json"

# Check if the files exist
if os.path.exists(titan_path):
    print('File "mercuryData_ไททัน.json" exists')
else:
    print('File "mercuryData_ไททัน.json" not found')

if os.path.exists(boss_path):
    print('File "mercuryData_บอสตาล.json" exists')
else:
    print('File "mercuryData_บอสตาล.json" not found')

if os.path.exists(yok_path):
    print('File "mercuryData_น้องหยก.json" exists')
else:
    print('File "mercuryData_น้องหยก.json" not found')

File "mercuryData_ไททัน.json" exists
File "mercuryData_บอสตาล.json" exists
File "mercuryData_น้องหยก.json" exists


In [13]:
#@title Filter null data
import json

# Assuming your data is stored in a JSON file called 'titan.json'

loaded_successfully = False
while not loaded_successfully:
    # try:
        with open(titan_path, 'r') as file:
            titan_data = json.load(file)
        loaded_successfully = True
    # except (FileNotFoundError, json.JSONDecodeError) as e:
    #     print("Error loading Titan JSON file:", e)
    #     print("Retrying...")

# Open the JSON file and load its contents
# with open(titan_path, 'r') as file:
#     titan_data = json.load(file)

with open(boss_path, 'r') as file:
    boss_data = json.load(file)

with open(yok_path, 'r') as file:
    yok_data = json.load(file)

# print(len(titan_data))

#filter top_words = null
filtered_titan = [titan for titan in titan_data if 'meta' in titan and 'top_words' in titan['meta'] and titan['meta']['top_words'] is not None]
filtered_boss = [boss for boss in boss_data if 'meta' in boss and 'top_words' in boss['meta'] and boss['meta']['top_words'] is not None]
filtered_yok = [yok for yok in yok_data if 'meta' in yok and 'top_words' in yok['meta'] and yok['meta']['top_words'] is not None]
# print(len(filtered_data))
print('(titan_size) before -> {} after -> {}'.format(len(titan_data), len(filtered_titan)) )
print('(boss_size) before -> {} after -> {}'.format(len(boss_data), len(filtered_boss)) )
print('(yok_size) before -> {} after -> {}'.format(len(yok_data), len(filtered_yok)) )

(titan_size) before -> 2746 after -> 2744
(boss_size) before -> 533 after -> 533
(yok_size) before -> 1138 after -> 1138


In [14]:
#@title Spilt ['doc_id'] and ['top_words'] to other dataframe
import pandas as pd

filtered_data = {
    'titan': filtered_titan,
    'boss': filtered_boss,
    'yok': filtered_yok
}

# Create a dictionary to store id_topWords DataFrames
id_topWords = {}

for identifier, data in filtered_data.items():
    id_topWords[identifier] = pd.DataFrame()

    for titan in data:
        doc_id = titan['doc_id']
        top_words = titan['meta']['top_words']
        new_row = {'doc_id': doc_id, 'top_word': top_words}
        id_topWords[identifier] = id_topWords[identifier].append(new_row, ignore_index=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-14-732e18c8cf2d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_topWords[identifier] = id_topWords[identifier].append(new_row, ignore_index=True)
<ipython-input-14-732e18c8cf2d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_topWords[identifier] = id_topWords[identifier].append(new_row, ignore_index=True)
<ipython-input-14-732e18c8cf2d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_topWords[identifier] = id_topWords[identifier].append(new_row, ignore_index=True)
<ipython-input-14-732e18c8cf2d>:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  id_topWord

In [5]:
#@title Sorted['top_words'] by alphabet th-en
import icu

collator = icu.Collator.createInstance(icu.Locale("th_TH.UTF-8"))

def sort_words(text):
    words = text.split(',')
    sorted_words = sorted(words, key=collator.getSortKey)
    return ', '.join(sorted_words)


df_titan = id_topWords['titan']
df_boss = id_topWords['boss']
df_yok = id_topWords['yok']

# Apply the sorting function to the 'top_word' column
df_titan['top_word'] = df_titan['top_word'].apply(sort_words)
df_boss['top_word'] = df_boss['top_word'].apply(sort_words)
df_yok['top_word'] = df_yok['top_word'].apply(sort_words)

In [ ]:
import pandas as pd

def aggregate_doc_ids_by_topwords(df):
    # Create a new DataFrame to store the results
    result_df = pd.DataFrame(columns=['top_word', 'doc_id'])

    # Iterate over each index in the original DataFrame
    for i in range(len(df)):
        current_top_word = df.loc[i, 'top_word']

        # Check if the current top_word value already exists in the new DataFrame
        matching_row = result_df[result_df['top_word'] == current_top_word]

        if len(matching_row) > 0:
            # If a matching row exists, append the doc_id to the existing row with a comma separator
            result_df.loc[matching_row.index[0], 'doc_id'] += f", {df.loc[i, 'doc_id']}"
        else:
            # If no matching row exists, create a new row with the current top_word and doc_id
            result_df = result_df.append({'top_word': current_top_word, 'doc_id': df.loc[i, 'doc_id']}, ignore_index=True)

    return result_df

# Apply the function to each DataFrame
sameTopWords_titan = aggregate_doc_ids_by_topwords(df_titan)
sameTopWords_boss = aggregate_doc_ids_by_topwords(df_boss)
sameTopWords_yok = aggregate_doc_ids_by_topwords(df_yok)

##check the result of aggregate_doc_ids_by_topwords

In [7]:
sameTopWords_titan.head()

,top_word,doc_id
0,"ซาก, ไททัน, ไททานิก, ผู้โดยสาร, เรือ, เรือดำน้...",IQHM0WW7OJIG7F4VEWRZVCIYVNOP8UKI
1,"คอนเสิร์ต, คาร์, ดิ, ไททัน, เรือดำน้ำ, สูญหาย,...","IQHM0WWECXXP0L3VVZX8EQJGV9WUDOIB, IQHM0WWAE9A7..."
2,"การยุบ, การระเบิด, ตัวถัง, เป็นจุณ, มิลลิวินาท...",IQHM0WW2RX6AUGUVO651JPCJ4GY0Z485
3,"คาเมรอน, ดำน้ำ, ทะเล, ไททัน, ไททานิก, ยาน, รัช...",IQHM0WWE40HEKUNW4AHFTLSHHPO885BZ
4,"การระเบิด, ดับ, ถัง, เพลิง, ระเบิด, แรงดัน, ลู...",IQHM0WW7PGC3X0U6RK1X65QZ2UGOZ8UB


In [8]:
sameTopWords_boss.head()

,top_word,doc_id
0,"ตาล, บัญชี, ประกันตัว, ผู้ต้องหา, ฝากขัง, พงษ์...",IQHM0WW4PL4CA9NHYDH550W6PNEL8HJ8
1,"ญาติ, ตาล, บอส, ผู้ต้องหา, ฝากขัง, พงษ์ศิริ, พ...","IQHM0WW8MQD7YD3Z1ZNKESBC8AQ0I84X, IQHM0WWDL0W8..."
2,"ตาล, บอส, ผู้ต้องหา, ฝากขัง, พงษ์ศิริ, พนัน, ม...",IQHM0WW0KOPX2N8U8ZQJEV4E6IOQ89F7
3,"โกศ, ข่าว, โจ๊ก, ปาท่องโก๋, พลิ้ว, พวงมาลา, มท...","IQHM0WW8ZNBO7UZGQHLC8OJOXXZ2F3N9, IQHL0WWAKOF3..."
4,"ใจไม้ไส้ระกำ, ตาล, พนัน, ฟอกเงิน, รองบ่อน, ระส...",IQHL0WW92BJFTVLQVRZAFSE1ZBW64QZK


In [9]:
sameTopWords_yok.head()

,top_word,doc_id
0,"น้อง, นักเรียน, รู้สึก, โรงเรียน, ศิษย์เก่า, ห...","IQHM0WW7V332QAEJHIQTMEN2L2F39RJA, IQHM0WW0BUGE..."
1,"น้อง, น้องๆ, นักเรียน, แบบนี้, ผม, รู้สึก, โรง...","IQHM0WW0TRUIQU3GJV5VTJNVNEX4EDGI, IQHM0WWBBN30..."
2,"ข้อ, คนแก่, คลิป, ดินแดน, ธา, พิ, มัลลิกา, สตอ...","IQHM0WWBIN4HKAMVODL5TPRPVK5N7EIJ, IQHM0WW41438..."
3,"คลิป, แชร์, ตอบ, น้อง, แบบนี้, แม่, หยก, แห่, ...",IQHM0WW2YPFSH9TL7LFZYW1J84HNTN29
4,"กฎหมาย, ข้อ, คลิป, เคลียร์, ดินแดน, ธา, พิ, มั...",IQHM0WW5LZHBZLHUL7PSDGV0CU1YDNMR


In [10]:
#@title save dataframe to .json
dataframes = [sameTopWords_titan, sameTopWords_boss, sameTopWords_yok]

for i, df in enumerate(dataframes):
    file_name = f"Grouped_TopWords_{['titan', 'boss', 'yok'][i]}.json"
    df.to_json(file_name)